In [1]:
from pulp import *
import numpy as np
import math
import time
import matplotlib.pyplot as plt
from collections import deque 
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show


In [2]:
# Using the bound in lemma 4.1 in the paper
# NOT constraining g(x, y) = (1 + h(x) - h(y))/2

# This is the discretization parameter
# Discretizes [0,1]^2 into an n-by-n grid
# Change this for finer/coarser discretizations
n = 50

prob = LpProblem("problem", LpMaximize)

t = LpVariable("t", lowBound=0, upBound=1)
prob += t

a = [k/n for k in range(0, n+1)]

# g(x, y) for x, y in discretized grid
g_vars = {(i,j): LpVariable(cat=LpContinuous, 
               lowBound=0, upBound=1, 
               name="g_{0}_{1}".format(i, j)) 
          for i in range(0, n+1) for j in range(0, n+1)}

## g(x, y) increasing in x
for j in range(0, n+1):
    for i in range(1, n+1):
        prob += g_vars[i, j] - g_vars[i-1, j] >= 0, "inceasing_constraint_{0}_{1}".format(i, j)
        
## g(x, y) decreasing in y
for i in range(0, n+1):
    for j in range(1, n+1):
        prob += g_vars[i, j] - g_vars[i, j-1] <= 0, "decreasing_constraint_{0}_{1}".format(i,j)
    
## dg(x, y)/dy >= g(x,y) - 1
for i in range(n):
    for j in range(n):
        prob += n*(g_vars[i, j+1] - g_vars[i, j]) >= g_vars[i+1, j]-1, "derivative_constraint_{0}_{1}".format(i, j)
# i=n; 
for j in range(n):
    prob += n*(g_vars[n, j+1] - g_vars[n, j]) >= g_vars[n, j]-1, "derivative_constraint_{0}_{1}".format(n, j)

    
## dg(x, y)/dx <= g(x,y) 
for j in range(n):
    for i in range(n):
        prob += n*(g_vars[i+1, j] - g_vars[i, j]) <= g_vars[i,j+1], "derivative_constraint2_{0}_{1}".format(i,j)
# j = n
for i in range(n):
    prob += n*(g_vars[i+1, n] - g_vars[i, n]) <= g_vars[i,n], "derivative_constraint2_{0}_{1}".format(i,n)
    
## g(1, y) - g(x, y) >= g(1, y') - g(x, y') for all x, y' > y
for i in range(0,n+1):
    for j in range(0, n+1):
        for k in range(j, n+1):
            prob += g_vars[n, j] - g_vars[i, j] >= g_vars[n, k] - g_vars[i, k]
        
## s(gamma, tau, y) represents min_{theta <= gamma} (1-g(theta, y)) + int_0^theta g(x, y)dx + int_theta^gamma g(x, tau)dx
s_vars = {(i,j,k): LpVariable(cat=LpContinuous,
                             lowBound=0,
                             upBound=1,
                             name="s_{0}_{1}_{2}".format(i,j,k))
         for i in range(0, n+1) for j in range(0, n+1) for k in range(0, j+1)}

# approximate integrals by left sums so as to get a lower bound
for i in range(0, n+1):
    for j in range(0, n+1):
        for k in range(0, j+1):
            for l in range(0, i+1):
                prob += s_vars[i,j,k] <= (1 - g_vars[l, k]
                                           + (1/n)*lpSum(g_vars[d,k] for d in range(0,l))
                                           + (1/n)*lpSum(g_vars[d,j] for d in range(l,i)))
    
# t = min{(1-a)(1-b) + (1-b)int_0^a g(x, b)dx + int_0^b min_c{g(x, c) + int_0^c g(y, x)dy + int_c^a g(y, b)dy}dx}
# where 0 <= a, b <= 1, c <= a, and g(x, y) = (1+h(x)-h(y))/2
# let s(a, b, x) = min_c{g(x, c) + int_0^c g(y, x)dy + int_c^a g(y, b)dy}
# so t = min{(1-a)(1-b) + (1-b)int_0^a g(x, b)dx + int_0^b s(a, b, x) dx}
for i in range(0, n+1):
    for j in range(0, n+1):
        prob += t <= ((1-a[i])*(1-a[j]) 
                       + (1-a[j])*(1/n)*lpSum(g_vars[l, j] for l in range(0, i))
                       + (1/n)*lpSum(s_vars[i,j,k] for k in range(0, j))) # unclear whether this is monotonic in x

In [3]:
# Solver can be changed
prob.solve(solver=GUROBI())

Using license file /Users/Prince/gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 1838703 rows, 70228 columns and 62472503 nonzeros
Model fingerprint: 0x9a0bd0c9
Coefficient statistics:
  Matrix range     [4e-04, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e-04, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 0 rows and 0 columns (presolve time = 14s) ...
Presolve removed 0 rows and 3876 columns (presolve time = 15s) ...
Presolve removed 2601 rows and 68901 columns (presolve time = 32s) ...
Presolve removed 2601 rows and 68953 columns (presolve time = 53s) ...
Presolve removed 2601 rows and 68953 columns (presolve time = 98s) ...
Presolve removed 2601 rows and 71554 columns (presolve time = 122s) ...
Presolve removed 2601 rows and 71554 columns (presolve time = 125s) ...
Presolve removed 2601 ro

1

In [4]:
# Objective value 
value(prob.objective)

0.6626421779818364

In [ ]:
g_values = np.zeros((n+1,n+1))
for i in range(n+1):
    for j in range(n+1):
        g_values[i,j] = value(g_vars[i,j])

In [ ]:
np.savetxt("g_values_{0}.txt".format(n))